In [1]:
# imports 
import os
import time
import datetime
import json
import gc

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
#import seaborn as sns
from tqdm import tqdm_notebook

from sklearn import metrics

from itertools import product
import json

In [2]:
dataDirectory = 'data'

In [3]:
#train = pd.read_csv(dataDirectory+'/train.csv')
structures = pd.read_csv(dataDirectory+'/structures.csv')

In [4]:
structures.head()

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397


#### See if there is a dominant 2D plane in the 3d molecular structure that captures most of the variance. If it is true we can reduce the dimesntionality and potentially use 2D convolution and train model with a lot less parameters.

In [ ]:
X = structures[['x','y','z']].values

pca = PCA(n_components=3)
pca.fit(X)  

print(pca.explained_variance_ratio_) 

#### this is not the case. Even in the orthogonal basis space of 3 dimensions all dimensions contribute significantly to variance. So I will stick with three dimensions.

In [14]:
oneHotDict = dict(zip(list(set(structures.atom)),[[1,0,0,0,0],[0,1,0,0,0],[0,0,1,0,0],[0,0,0,1,0],[0,0,0,0,1]]))

In [5]:
## distinct molecules
Molecules = list(set(structures.molecule_name))
len(Molecules)

130775

In [8]:
# Molecule Files already stored in earlier run
storedMol = os.listdir('data/images/')
storedMol = [s.split('.')[0] for s in storedMol]
print(len(storedMol))
storedMol[:5]

30828


['dsgdb9nsd_125883',
 'dsgdb9nsd_058319',
 'dsgdb9nsd_053209',
 'dsgdb9nsd_020868',
 'dsgdb9nsd_100369']

In [12]:
## Exclude the molecules already processed
Molecules = list(set(Molecules).difference(set(storedMol)))
len(Molecules)

99951

In [ ]:
#imgDict = {}
counter = 0
for mol in Molecules:
    #print(mol)
    img = np.zeros((20,20,20,6))
    stDf = structures[structures.molecule_name == mol]
    indexes = ((stDf[['x','y','z']].values+9.5)).astype(int)
    channel = np.array(list(stDf.atom.apply(lambda x: oneHotDict[x]).values))
    
    for i in range(indexes.shape[0]):
        img[indexes[i][0],indexes[i][1],indexes[i][2],0:5] = channel[i]
    
    
    np.save('data/images/'+str(mol),img)

    #imgDict[mol] = img.tolist()
    
    
    counter+=1
    if counter % 100 == 0:
        print( str((counter/len(Molecules))*100)+" %")


0.10004902402177068 %
0.20009804804354137 %
0.300147072065312 %
0.40019609608708273 %
0.5002451201088534 %
0.600294144130624 %
0.7003431681523947 %
0.8003921921741655 %
0.9004412161959361 %
1.0004902402177067 %
1.1005392642394773 %
1.200588288261248 %
1.3006373122830186 %
1.4006863363047894 %
1.50073536032656 %
1.600784384348331 %
1.7008334083701016 %
1.8008824323918722 %
1.9009314564136428 %
2.0009804804354134 %
2.101029504457184 %
2.2010785284789547 %
2.3011275525007253 %
2.401176576522496 %
2.5012256005442666 %
2.601274624566037 %
2.701323648587808 %
2.801372672609579 %
2.9014216966313495 %
3.00147072065312 %
3.1015197446748908 %
3.201568768696662 %
3.3016177927184325 %
3.401666816740203 %
3.5017158407619737 %
3.6017648647837444 %
3.701813888805515 %
3.8018629128272856 %
3.9019119368490562 %
4.001960960870827 %
4.1020099848925975 %
4.202059008914368 %
4.302108032936139 %
4.402157056957909 %
4.50220608097968 %
4.602255105001451 %
4.702304129023221 %
4.802353153044992 %
4.902402177066

In [ ]:
## code adopted from: https://www.kaggle.com/shivamb/3d-convolutions-understanding-use-case
def getCNN():
    ## input layer
    input_layer = Input((40, 40, 40, 6))

    ## convolutional layers
    conv_layer1 = Conv3D(filters=8, kernel_size=(3, 3, 3), activation='relu')(input_layer)
    conv_layer2 = Conv3D(filters=16, kernel_size=(3, 3, 3), activation='relu')(conv_layer1)

    ## add max pooling to obtain the most imformatic features
    pooling_layer1 = MaxPool3D(pool_size=(2, 2, 2))(conv_layer2)

    conv_layer3 = Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu')(pooling_layer1)
    conv_layer4 = Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu')(conv_layer3)
    pooling_layer2 = MaxPool3D(pool_size=(2, 2, 2))(conv_layer4)

    ## perform batch normalization on the convolution outputs before feeding it to MLP architecture
    pooling_layer2 = BatchNormalization()(pooling_layer2)
    flatten_layer = Flatten()(pooling_layer2)

    ## create an MLP architecture with dense layers : 4096 -> 512 -> 10
    ## add dropouts to avoid overfitting / perform regularization
    dense_layer1 = Dense(units=2048, activation='relu')(flatten_layer)
    dense_layer1 = Dropout(0.4)(dense_layer1)
    dense_layer2 = Dense(units=512, activation='relu')(dense_layer1)
    dense_layer2 = Dropout(0.4)(dense_layer2)
    output_layer = Dense(units=1, activation='linear')(dense_layer2)

    ## define the model with input layer and output layer
    model = Model(inputs=input_layer, outputs=output_layer)
    return model

In [ ]:
def prepareCNNData(df):
    